<a href="https://colab.research.google.com/github/arkeyan/Indicators/blob/main/Kompozit_Banker_Dip_Sinyal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed  backtesting
!pip install tradingview-screener==2.5.0

import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
from backtesting import Backtest, Strategy
import warnings

warnings.simplefilter(action='ignore')


# Fonksiyonlar
def xsa(src, length, weight):
    sumf = np.zeros_like(src)
    ma = np.zeros_like(src)
    out = np.zeros_like(src)

    for i in range(length, len(src)):
        sumf[i] = np.nan_to_num(sumf[i - 1]) - np.nan_to_num(src[i - length]) + src[i]
        ma[i] = np.nan if np.isnan(src[i - length]) else sumf[i] / length
        out[i] = ma[i] if np.isnan(out[i - 1]) else (src[i] * weight + out[i - 1] * (length - weight)) / length

    return out

def ema(src, span):
    return src.ewm(span=span, adjust=False).mean()


def Banker_Fund_Trend(data):
    df = data.copy()
    close = df['close']
    low = df['low']
    high = df['high']
    open_ = df['open']
    fundtrend = np.zeros_like(close)
    part_1 = (close - low.rolling(window=27).min()) / (high.rolling(window=27).max() - low.rolling(window=27).min()) * 100
    fundtrend = ((3 * xsa(part_1, 5, 1) - 2 * xsa(xsa(part_1, 5, 1), 3, 1) - 50) * 1.032 + 50)

    typ = (2 * close + high + low + open_) / 5
    lol = low.rolling(window=34).min()
    hoh = high.rolling(window=34).max()

    bullbearline = ema((typ - lol) / (hoh - lol) * 100, 13)
    bankerentry = (fundtrend > bullbearline) & (bullbearline < 25)

    df['fundtrend'] = fundtrend
    df['bullbearline'] = bullbearline
    df['bankerentry'] = bankerentry.astype(int)

    return df


# Periyot Seçimi
PERIODS = {
    '1': ('Günlük', Interval.in_daily),
    '2': ('Haftalık', Interval.in_weekly),
    '3': ('Aylık', Interval.in_monthly)
}

print("Lütfen bir tarama periyodu seçin:")
for key, (desc, _) in PERIODS.items():
    print(f"{key}: {desc}")
period_choice = input("Seçiminiz: ")

if period_choice not in PERIODS:
    print("Geçersiz seçim, günlük periyot kullanılacak.")
    period_choice = '1'

selected_period, interval = PERIODS[period_choice]

# TvDatafeed ve Hisse Verileri
tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# Raporlama
Titles = ['Hisse Adı', 'Son Fiyat', 'Dip Sinyali']
df_signals = pd.DataFrame(columns=Titles)

for hisse in Hisseler:
    try:
        data = tv.get_hist(symbol=hisse, exchange='BIST', interval=interval, n_bars=377)
        dataref = tv.get_hist(symbol='XU100', exchange='BIST', interval=interval, n_bars=377)
        data = data.reset_index()
        dataref = dataref.reset_index()
        result = data.copy()
        columns_to_divide = ['open', 'high', 'low', 'close', 'volume']
        for column in columns_to_divide:
            result[column] = 100 * (data[column] / dataref[column])
        Banker = Banker_Fund_Trend(result)
        Banker.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        Banker.set_index('datetime', inplace=True)

        Buy = False
        Signals = Banker.tail(2)
        Signals = Signals.reset_index()
        Buy = Signals.loc[0, 'bankerentry'] == 0 and Signals.loc[1, 'bankerentry'] == 1
        Last_Price = Signals.loc[1, 'Close']
        L1 = [hisse, round(Last_Price, 2), Buy]
        df_signals.loc[len(df_signals)] = L1
        print(L1)
    except Exception as e:
        print(f"{hisse} için hata: {e}")

df_True = df_signals[df_signals['Dip Sinyali'] == True]
print(df_True)


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-1c43nisv
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-1c43nisv
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.9/187.9 kB 1.8 MB/s eta 0:00:00
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=8f1c7c931e87300b1b7a4fe7d35a03bc728bbccea04739c0dee1947d016ef6e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-9xmhq6b5/wheels/0a/ba/99/b27476fd1e4caf0dd70445cdc6798195d3b90005a1501a12f7
Successfully built tvdatafeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 878.9 kB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:54: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Lütfen bir tarama periyodu seçin:
1: Günlük
2: Haftalık
3: Aylık
Seçiminiz: 1


['A1CAP', 0.04, False]
['ACSEL', 0.98, False]
['ADEL', 0.28, True]
['ADESE', 0.02, False]
['ADGYO', 0.29, False]
['AEFES', 1.73, False]
['AFYON', 0.15, False]
['AGESA', 1.51, False]
['AGHOL', 3.2, False]
['AGROT', 0.09, False]
['AGYO', 0.07, False]
['AHGAZ', 0.19, False]
['AHSGY', 0.24, False]
['AKBNK', 0.72, False]
['AKCNS', 1.67, False]
['AKENR', 0.12, False]
['AKFGY', 0.02, False]
['AKFIS', 0.27, False]
['AKFYE', 0.17, False]
['AKGRT', 0.06, False]
['AKMGY', 2.28, False]
['AKSA', 0.13, False]
['AKSEN', 0.34, False]
['AKSGY', 0.07, False]
['AKSUE', 0.12, False]
['AKYHO', 0.04, False]
['ALARK', 0.78, True]
['ALBRK', 0.07, False]
['ALCAR', 8.77, False]
['ALCTL', 1.14, False]
['ALFAS', 0.53, False]
['ALGYO', 0.18, False]
['ALKA', 0.07, False]
['ALKIM', 0.16, False]
['ALKLC', 0.3, False]
['ALMAD', 0.08, False]
['ALTIN', 0.36, False]
['ALTNY', 0.72, False]
['ALVES', 0.32, True]
['ANELE', 0.14, False]
['ANGEN', 0.12, False]
['ANHYT', 1.0, False]
['ANSGR', 1.04, False]
['APBDL', 0.24, False